In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser

In [ ]:
#function to rename columns of tables where rushing is first
def rushcolumnsrename(df,prefix):
    df.rename(columns={'G':f'{prefix}games',
                       'Att':f'{prefix}RushAtt',
                        'Yds':f'{prefix}RushYds',
                        'Avg':f'{prefix}YPA',
                        'TD':f'{prefix}RushTds',
                        'Rec':f'{prefix}Rec',
                        'Yds.1':f'{prefix}RecYds',
                        'Avg.1':f'{prefix}YPC',
                        'TD.1':f'{prefix}RecTds',
                        'Plays':f'{prefix}Plays',
                        'Yds.2':f'{prefix}TotalYds',
                        'Avg.2':f'{prefix}YPT',
                        'TD.2':f'{prefix}TotalTds'}, inplace = True)
    return df

In [ ]:
#function to rename columns of tables where receiving is first
def reccolumnsrename(df,prefix):
    df.rename(columns={'G':f'{prefix}games',
                        'Rec':f'{prefix}Rec',
                        'Yds':f'{prefix}RecYds',
                        'Avg':f'{prefix}YPC',
                        'TD':f'{prefix}RecTds',
                        'Att':f'{prefix}RushAtt',
                        'Yds.1':f'{prefix}RushYds',
                        'Avg.1':f'{prefix}YPA',
                        'TD.1':f'{prefix}RushTds',
                        'Plays':f'{prefix}Plays',
                        'Yds.2':f'{prefix}TotalYds',
                        'Avg.2':f'{prefix}YPT',
                        'TD.2':f'{prefix}TotalTds'}, inplace = True)
    return df

In [ ]:
#function to append 0's if someone has less than 3 years of stats- for 2 guys.  Don't want to eliminate them yet
def append0less3(df):
    df['dr_2games']=0,
    df['dr_2Rec']=0,
    df['dr_2RecYds']=0,
    df['dr_2YPC']=0,
    df['dr_2RecTds']=0,
    df['dr_2RushAtt']=0,
    df['dr_2RushYds']=0,
    df['dr_2YPA']=0,
    df['dr_2RushTds']=0,
    df['dr_2Plays']=0,
    df['dr_2TotalYds']=0,
    df['dr_2YPT']=0,
    df['dr_2TotalTds']=0,
    df['dr_3games']=0,
    df['dr_3Rec']=0,
    df['dr_3RecYds']=0,
    df['dr_3YPC']=0,
    df['dr_3RecTds']=0,
    df['dr_3RushAtt']=0,
    df['dr_3RushYds']=0,
    df['dr_3YPA']=0,
    df['dr_3RushTds']=0,
    df['dr_3Plays']=0,
    df['dr_3TotalYds']=0,
    df['dr_3YPT']=0,
    df['dr_3TotalTds']=0
    return df

In [ ]:
#function to append 0's if someone has less than 4 years of stats. Note: 2 years and 1 career row
def append0less4(df):
    df['dr_3games']=0,
    df['dr_3Rec']=0,
    df['dr_3RecYds']=0,
    df['dr_3YPC']=0,
    df['dr_3RecTds']=0,
    df['dr_3RushAtt']=0,
    df['dr_3RushYds']=0,
    df['dr_3YPA']=0,
    df['dr_3RushTds']=0,
    df['dr_3Plays']=0,
    df['dr_3TotalYds']=0,
    df['dr_3YPT']=0,
    df['dr_3TotalTds']=0
    return df

In [ ]:
# Function to parse out individuals college stats and store in a single row with appropriate labels for career,
# draft_1 (year before drafted (Note: draft year is 1 year after season), draft_2 (2 years before drafted),
# draft_3 (years before drafted)

#sample containing a person with 3, 4 and 5 rows for testing
sample = ['Derrick Williams', 'Cordarrelle Patterson','Jeremy Maclin']

def stats1row(indivs,df,colRename):
    #result is returned as a list created here
    rows = []
    #loop through the indexes of a list of individuals
    for i in range(len(indivs)):
        years=[]
        
        #store the individual
        player = indivs[i]
        
        #limit dataframe to results with that individual
        playerdf = df[df['Player']==indivs[i]]
#         print(len(playerdf))
        #store the total rows with stats
        years = playerdf['Year'].unique()
        #Get career data in a dataframe
        career = playerdf[playerdf['Year']==years[-1]]
        #rename columns
        car_row = colRename(career,'career')
        #drop columns
        car_row.drop(columns = ['Year', 'School', 'Conf', 'Class', 'Pos'], inplace = True)
        #get dataframe of 1 year prior to draft
        draft_1 = playerdf[playerdf['Year']==years[-2]]
        #rename columns
        lastyr_row = colRename(draft_1,'dr_1')

        #new dataframe of career and 1 year prior merged
        career_one_row = car_row.merge(lastyr_row, on='Player',how = 'inner')
        
        #check if df has less than 3 rows
        if len(playerdf)<3:
            #if so append zeroes for those columns that can't be appended
            career_one_row = append0less3(career_one_row)
        else:
            #get dataframe of 2 year prior to draft
            draft_2 = playerdf[playerdf['Year']==years[-3]]
            last2yr_row = colRename(draft_2,'dr_2')
            last2yr_row.drop(columns = ['Year', 'School', 'Conf', 'Class', 'Pos'], inplace = True)

            #merge 2 years prior
            career_one_row= career_one_row.merge(last2yr_row, on = 'Player', how = 'inner')
            
            #check if df has less than 4 rows
            if len(playerdf)<4:
                #if so append zeroes for those columns that can't be appended
                career_one_row = append0less4(career_one_row)
            else:
                #get dataframe of 3 years prior to draft
                draft_3 = playerdf[playerdf['Year']==years[-4]]
                last3yr_row = colRename(draft_3,'dr_3')
                last3yr_row.drop(columns = ['Year', 'School', 'Conf', 'Class', 'Pos'], inplace = True)

                #merge 3 years prior
                career_one_row= career_one_row.merge(last3yr_row, on = 'Player', how = 'inner')
#             print('--------------------')

        rows.append(career_one_row)
    return rows
#     print(lastyr_row)

In [ ]:
#Overall the purpose of this is to scrape information on players and store names and urls of players that were drafted
# and they have college stats on record.  Note: this made me limit my exploration to D1 players because D2 player statistics
# were much more individualized.  In the future I would like to expand my dataset to include those individuals so I have
# stored the record of those individuals that were drafted, but didn't have college stats

#spin up web browser to visit different combine results pages by year
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#Create a list of years of combine stats that I will gather
years = ['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']
#list to store raw dataframes in case I need to reference later
dfs=[]
#place to store dictionaries of players and the url's to their statistics
links = []
#list to store the information of players that don't have links to stats
p_no_stats = []
for yr in years:
    print(yr)
    print('-------------------------------')
    #insert years into url, visit url, and turn into a soup object
    url = f"https://www.pro-football-reference.com/play-index/nfl-combine-results.cgi?request=1&year_min={yr}&year_max={yr}&height_min=60&height_max=82&weight_min=120&weight_max=400&pos%5B%5D=WR&show=all&order_by=year_id"
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    #separate out all html elements with label data-stat = college which corresponds to the number of indivs on the page
    a= soup.find_all(attrs={"data-stat": "college"})
    
    #loop through by index to check all rows of data stored in the combine table
    for i in range(len(a)):
        
        #for each grab and store the element that has the player name
        name = soup.find_all(attrs ={"data-stat":"player"})
        #store the value that indicates if a player was drafted, the round, pick, and team
        draft = soup.find_all(attrs = {'data-stat':'draft_info'})
        
        #assign the player name to variable player
        player = name[i].text
        #assign the drafted text to variable drafted
        drafted = draft[i].text
        
        #check if length of drafted is 0, which indicates that there is no record or the player wasn't drafted
        if len(drafted) == 0:
            #print out the player name to check functioning. loop returns
#             print(f'{player} was not drafted')
        
        #if a player was drafted do the following
        else:
            #try catch to attempt to get the link to college stats
            try: 
                #get url of player
                link = a[i].find('a')['href']
                #print and store a combination of player and url
#                 print(f'{player} drafted college stats here: {link}')
                links.append({'player':player,'url':link})
            #handling players without a url    
            except:
                #separate out header rows so they are not stored
                if player == 'Player':
                    pass
#                     print('header row')
                #store the info of players that were drafted, but didn't have college stats
                else:
#                     print(f"{player} No college Stats")
                    p_no_stats.append(player)
    print("Next Year")


In [ ]:
#variable storing a few players name and url's for creating, troubleshooting, and maybe adjustments later on
couplelinks = [{'player': 'Lynn Bowden', 'url': 'https://www.sports-reference.com/cfb/players/lynn-bowden-jr-1.html'},
{'player': 'Brandon Aiyuk', 'url': 'https://www.sports-reference.com/cfb/players/brandon-aiyuk-1.html'},
               {'player': 'Chase Claypool', 'url': 'https://www.sports-reference.com/cfb/players/chase-claypool-1.html'}]

In [ ]:
#larger subset of 2009 data for larger testing, troubleshooting
links2009 = [{'player': 'Derrick Williams', 'url': 'https://www.sports-reference.com/cfb/players/derrick-williams-2.html'}, {'player': 'Mike Wallace', 'url': 'https://www.sports-reference.com/cfb/players/mike-wallace-1.html'}, {'player': 'Tiquan Underwood', 'url': 'https://www.sports-reference.com/cfb/players/tiquan-underwood-1.html'}, {'player': 'Patrick Turner', 'url': 'https://www.sports-reference.com/cfb/players/patrick-turner-1.html'}, {'player': 'Mike Thomas', 'url': 'https://www.sports-reference.com/cfb/players/mike-thomas-9.html'}, {'player': 'Brandon Tate', 'url': 'https://www.sports-reference.com/cfb/players/brandon-tate-1.html'}, {'player': 'Sammie Stroughter', 'url': 'https://www.sports-reference.com/cfb/players/sammie-stroughter-1.html'}, {'player': 'Demetrius Byrd', 'url': 'https://www.sports-reference.com/cfb/players/demetrius-byrd-1.html'}, {'player': 'Brian Robiskie', 'url': 'https://www.sports-reference.com/cfb/players/brian-robiskie-1.html'}, {'player': 'Hakeem Nicks', 'url': 'https://www.sports-reference.com/cfb/players/hakeem-nicks-1.html'}, {'player': 'Louis Murphy', 'url': 'https://www.sports-reference.com/cfb/players/louis-murphy-1.html'}, {'player': 'Marko Mitchell', 'url': 'https://www.sports-reference.com/cfb/players/marko-mitchell-1.html'}, {'player': 'Kenny McKinley', 'url': 'https://www.sports-reference.com/cfb/players/kenny-mckinley-1.html'}, {'player': 'Mohamed Massaquoi', 'url': 'https://www.sports-reference.com/cfb/players/mohamed-massaquoi-1.html'}, {'player': 'Jeremy Maclin', 'url': 'https://www.sports-reference.com/cfb/players/jeremy-maclin-1.html'}, {'player': 'Manuel Johnson', 'url': 'https://www.sports-reference.com/cfb/players/manuel-johnson-1.html'}, {'player': 'Juaquin Iglesias', 'url': 'https://www.sports-reference.com/cfb/players/juaquin-iglesias-1.html'}, {'player': 'Darrius Heyward-Bey', 'url': 'https://www.sports-reference.com/cfb/players/darrius-heyward-bey-1.html'}, {'player': 'Percy Harvin', 'url': 'https://www.sports-reference.com/cfb/players/percy-harvin-1.html'}, {'player': 'Brian Hartline', 'url': 'https://www.sports-reference.com/cfb/players/brian-hartline-1.html'}, {'player': 'Brandon Gibson', 'url': 'https://www.sports-reference.com/cfb/players/brandon-gibson-1.html'}, {'player': 'Brooks Foster', 'url': 'https://www.sports-reference.com/cfb/players/brooks-foster-1.html'}, {'player': 'Jarett Dillard', 'url': 'https://www.sports-reference.com/cfb/players/jarett-dillard-1.html'}, {'player': 'Michael Crabtree', 'url': 'https://www.sports-reference.com/cfb/players/michael-crabtree-1.html'}, {'player': 'Austin Collie', 'url': 'https://www.sports-reference.com/cfb/players/austin-collie-1.html'}, {'player': 'Deon Butler', 'url': 'https://www.sports-reference.com/cfb/players/deon-butler-1.html'}, {'player': 'Kenny Britt', 'url': 'https://www.sports-reference.com/cfb/players/kenny-britt-1.html'}]

In [ ]:
#This entire chunck will convert all of the players information into a consistent format of 1 row of information and
#and combine into 1 dataframe of college stats.  There is clean up as well.

#Get a list of unique players
check= all_df['Player'].unique()

#interested in knowing the distribution of years played among players
players_stat_yrs = []
players = []
for i in range(len(check)):
    years=[]
    players.append(check[i])
    playerdf = all_df[all_df['Player']==check[i]]
    yrs_played = len(playerdf)
    players_stat_yrs.append(yrs_played)
    
#create data frame of results to understand number of years of stats of players
yearsofstats_df = pd.DataFrame({'players':players,'statyears':players_stat_yrs})
#Group by statyears to know the number of players with corresponding stat years
#Note the 9 is Mike Williams which is actually 2 players 1 from drafted by the Bucs and the other Stud from Clemson Go TIGERS!
print(yearsofstats_df.groupby('statyears').count())

#look at record of 2 players to see if valid to just exclude since it represents 2 players that only have 1 year of stats
years2 = yearsofstats_df[yearsofstats_df['statyears']==2]
print(years2)

#look at player with 9 rows.  It is actually Mike Williams which pertains to 2 players so creating separate dataframe below
years9 = yearsofstats_df[yearsofstats_df['statyears']==9]
print(years2)

#get all guys that are primarily receivers and played at a single school into a df with 1 row per player stats
rec_guys = stats1row(check,all_df,reccolumnsrename)

#create dataframe of the list of dataframes created with function
rec_final = pd.concat(rec_guys)
#check the length to validate the function worked properly
len(rec_final)
#Exclude Mike Williams
minus_mikew = rec_final[rec_final['Player']!='Mike Williams']
#save as CSV file
minus_mikew.to_csv("recfinal.csv")

# Get a dataframe consisting of only Mike Williams from Clemson Info
clem_mike_w = all_df[(all_df['Player']=='Mike Williams')&(all_df['School']=='Clemson')]
clem_mike_w_onerow=stats1row(['Mike Williams'],clem_mike_w,reccolumnsrename)
clem_mike_w_onerow[0]

#Get a dataframe consisting of only Mike Williams from Syracuse Info
syracuse_mike_w = all_df[(all_df['Player']=='Mike Williams')&(all_df['School']=='Syracuse')]
syracuse_mike_w_onerow=stats1row(['Mike Williams'],syracuse_mike_w,reccolumnsrename)
syracuse_mike_w_onerow[0]

#create dataframe of players collected with rushing stats first
rushing_first= pd.concat(p_no_recs_dfs)
# get unique list of player names to use in function
rush_players = rushing_first['Player'].unique()
#run dataframe through function to get 1 row per player
rush_df = stats1row(rush_players,rushing_first,rushcolumnsrename)
#Put togther list of dfs into 1
rush_to_rec_form = pd.concat(rush_df)
#Reorder the columns to join with the rec_guys dataframe
rush_final = rush_to_rec_form[['careergames', 'careerRec', 'careerRecYds', 'careerYPC', 'careerRecTds',
       'careerRushAtt', 'careerRushYds', 'careerYPA', 'careerRushTds',
       'careerPlays', 'careerTotalYds', 'careerYPT', 'careerTotalTds',
       'Player', 'Year', 'School', 'Conf', 'Class', 'Pos', 'dr_1games',
       'dr_1Rec', 'dr_1RecYds', 'dr_1YPC', 'dr_1RecTds', 'dr_1RushAtt',
       'dr_1RushYds', 'dr_1YPA', 'dr_1RushTds', 'dr_1Plays', 'dr_1TotalYds',
       'dr_1YPT', 'dr_1TotalTds', 'dr_2games', 'dr_2Rec', 'dr_2RecYds',
       'dr_2YPC', 'dr_2RecTds', 'dr_2RushAtt', 'dr_2RushYds', 'dr_2YPA',
       'dr_2RushTds', 'dr_2Plays', 'dr_2TotalYds', 'dr_2YPT', 'dr_2TotalTds',
       'dr_3games', 'dr_3Rec', 'dr_3RecYds', 'dr_3YPC', 'dr_3RecTds',
       'dr_3RushAtt', 'dr_3RushYds', 'dr_3YPA', 'dr_3RushTds', 'dr_3Plays',
       'dr_3TotalYds', 'dr_3YPT', 'dr_3TotalTds']]

#create df of all players that attended multiple schools
mult_schools = pd.concat(redo_dfs)
#get rid of subtotal rows by school
mult_schools['Year'].fillna('No Year',inplace=True)
mult_schools_final = mult_schools[mult_schools['Year']!= 'No Year']
#get unique list of players
mult_school_players = mult_schools_final['Player'].unique()
#convert to a single row
mult_schls = stats1row(mult_school_players,mult_schools_final,reccolumnsrename)
#concatinate list into 1 dataframe
mult_schools_one_row = pd.concat(mult_schls)
#save csv of mult_schools_data
mult_schools_one_row.to_csv("mult_schools.csv")

#Put together all dfs of players that played for a single team
all_college = pd.concat([minus_mikew,clem_mike_w_onerow[0],syracuse_mike_w_onerow[0],rush_to_rec_form,mult_schools_one_row])

#save to csv file- Commented out since file is final and used elsewhere
# all_college.to_csv("allcollegestat.csv")